In [8]:
from openai import OpenAI

# filepath: /path/to/your/file.py
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the OpenAI API key
openai_api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI()

In [9]:


client.files.create(
  file=open("dad_joke_reasoning_samples_test.jsonl", "rb"),
  purpose="fine-tune"
)

client.files.create(
  file=open("dad_joke_reasoning_samples_train.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-RupsWX4Jow3xkvbKASNVfH', bytes=72417, created_at=1739197688, filename='dad_joke_reasoning_samples_train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [10]:
files = client.files.list(purpose="fine-tune")

for file in files:
    print(file.filename.upper(),':',file.id)

DAD_JOKE_REASONING_SAMPLES_TRAIN.JSONL : file-RupsWX4Jow3xkvbKASNVfH
DAD_JOKE_REASONING_SAMPLES_TEST.JSONL : file-AUBGAikbAQqQu87UNL4xTB
DAD_JOKE_REASONING_SAMPLES_TRAIN.JSONL : file-W6ENYGH6B4PWEhAYb3SJ4e
DAD_JOKE_REASONING_SAMPLES_TEST.JSONL : file-CBpU2v53Y4VT86GWtqHcuC
DAD_JOKE_SAMPLES_TEST.JSONL : file-AkPWUJUXbsLLsdkETJrgW4
DAD_JOKE_SAMPLES_TRAIN.JSONL : file-78aGFZduAzCr7gNpULVkpD
DAD_JOKE_REASONING_SAMPLES_TRAIN.JSONL : file-XZkorTai3cwNRvj4t3Eaap
DAD_JOKE_REASONING_SAMPLES_TEST.JSONL : file-1fwP5mJBAVSJm1enXNHvWC
DAD_JOKE_SAMPLES_TEST.JSONL : file-14gtA38iuJ7sJgqBmwsyNC
DAD_JOKE_SAMPLES_TRAIN.JSONL : file-Jp47nPA2Eq2S9p7VXCEsWz


In [15]:
client.fine_tuning.jobs.create(
  training_file="file-XZkorTai3cwNRvj4t3Eaap",
  validation_file ="file-1fwP5mJBAVSJm1enXNHvWC",
  model="gpt-4o-2024-08-06",
  suffix="cold-warrior-reasoning"
)

FineTuningJob(id='ftjob-CS6URQHmSAxPW8Z2eKLLbO2Z', created_at=1739198765, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-RuBNDg5GaMqiVIMAjwK7becX', result_files=[], seed=206067273, status='validating_files', trained_tokens=None, training_file='file-XZkorTai3cwNRvj4t3Eaap', validation_file='file-1fwP5mJBAVSJm1enXNHvWC', estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix='cold-warrior-reasoning')

In [16]:
import time


while True:
    events = client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-CS6URQHmSAxPW8Z2eKLLbO2Z", limit=10)
    message = events.data[0].message
    print(message)
    if message == "The job has successfully completed":
        break
    

    time.sleep(20)

Fine-tuning job started
Fine-tuning job started
Fine-tuning job started
Fine-tuning job started
Fine-tuning job started
Fine-tuning job started
Fine-tuning job started
Fine-tuning job started
Fine-tuning job started
Fine-tuning job started
Fine-tuning job started
Step 1/144: training loss=1.59
Step 4/144: training loss=1.29
Step 9/144: training loss=1.38
Step 16/144: training loss=1.21
Step 30/144: training loss=1.37, validation loss=1.11
Step 41/144: training loss=1.19
Step 49/144: training loss=1.28
Step 60/144: training loss=1.00, validation loss=0.93
Step 75/144: training loss=1.21
Step 87/144: training loss=1.08
Step 96/144: training loss=0.81, full validation loss=1.09
Step 96/144: training loss=0.81, full validation loss=1.09
Step 108/144: training loss=0.91
Step 119/144: training loss=0.77
Step 131/144: training loss=0.94
Step 141/144: training loss=0.94
Step 144/144: training loss=1.14, validation loss=1.01, full validation loss=1.09
Step 144/144: training loss=1.14, validatio

In [17]:
# Retrieve the state of a fine-tune
dad_joke_model = client.fine_tuning.jobs.retrieve("ftjob-CS6URQHmSAxPW8Z2eKLLbO2Z").fine_tuned_model
print(dad_joke_model)

ft:gpt-4o-2024-08-06:personal:cold-warrior-reasoning:AzPUeQFT
